<h1 style="text-align: center;text-transform: uppercase;">Conversational Based Agent</h1>

<br>

In this project, you will build an end-to-end voice conversational agent, which can take a voice input audio line, and synthesize a response. The chatbot agent will be executed locally on your computer. 

<img width="600px" src="assets/siri.jpg">

This jupyter notebook is consists of the following parts:
1. __Speech Recognition:__ <br>In this part, you will create a speech recognition that can convert your voice into a text format.<br><br>
2. __Chatbot:__ <br>This is the core of your conversational based agent. You will build a chatbot that will answer your questions. <br><br>
3. __Text to Speech:__ <br>After getting the answer from your chatbot, it should be converted into a voice format and that is what you should create in this part. <br><br>
4. __Finalize your Conversational Based Agent:__ <br>At the very end step, you will put everything together and create your Conversational Based Agent.

<br>

# 1. Speech Recognition

---

<br>

# 2. Chatbot

---


In this part, you will create a deep learning based conversational agent. This agent will be able to interact with users and understand their questions. More specifically, you will start with loading the dataset, cleaning and preprocessing them, and then you will feed them into a neural network.

<br>

### 3.1. Load and Clean the Dataset

---

In this project, we have provided you with multiple dataset files. Each of these files contains conversations regarding a specific topic. For example, topics about humor, food, movies, science, history, etc. You can read the description of each dataset in below:

| Name of Dataset | Description |
| :----:| :----: |
| botprofile.yml | Personality of Your Chatbot |
| humor.yml | Joke and Humor |
| emotion.yml | Emotional Conversations |
| politics.yml | Political Conversations |
| ai.yml | General Questions about AI |
| computers.yml | Conversations about Computer |
| history.yml | Q&A about Historical Facts and Events |
| psychology.yml | Psychological Conversations |
| food.yml | Food Related Conversations. |
| literature.yml | Conversations about Different Books, Authors, Genres |
| money.yml | Conversations about Money, Investment, Economy |
| trivia.yml | Conversations that Have Small Values |
| gossip.yml | Gossipy Conversations |
| conversations.yml | Common Conversations |
| greetings.yml | Different Ways of Greeting |
| sports.yml | Conversations about Sports. |
| movies.yml | Conversation about Movies. |
| science.yml | Conversations about Science  |
| health.yml | Health Related Questions and Answers. |


Feel free to modify these datasets in the way you want the chatbot to behave. 

In [124]:
# Import the libraries
import yaml
import glob
import tqdm

In [47]:
# Function for loading all of the yml files
def load_dataset():
    
    # Initialize empty lists for questions and answers
    questions, answers = [], []
    
    # Get the list of all dataset names
    dataset_names = glob.glob("dataset/*.yml")
    
    # Iterate through each dataset name
    for i_dataset_name in tqdm.tqdm(dataset_names):
        
        # Load the dataset
        with open(i_dataset_name) as file:
            greeting = yaml.load(file)["conversations"]
            
        # Iterate through each conversation
        for i_conversation in greeting:
            
            # If length is two
            if len(i_conversation) == 2:
                
                # Append the question to 'questions' list
                questions.append(i_conversation[0])
                
                # Append the answer to 'answers' list
                answers.append(i_conversation[1])
            
            # If length is more than two
            elif len(i_conversation) > 2:
                
                # Iterate through each index
                for index in range (len(i_conversation)-1):
                    
                    # Append the question and answer
                    questions.append(i_conversation[0])
                    answers.append(i_conversation[index+1])
                    
    return questions, answers

In [48]:
# Get the questions and answers
questions, answers = load_dataset()

100%|██████████| 19/19 [00:00<00:00, 38.91it/s]


<br>

### 3.3. Data Preprocessing

---

After cleaning the dataset, you should preprocess the dataset by following the below steps:

1. Lower case the text.
2. Decontract the text (e.g. she's -> she is, they're -> they are, etc.).
3. Remove the punctuation (e.g. !, ?, $, %, #, @, ^, etc.).
4. Tokenization.
5. Pad the sequences to be the same length.

In [63]:
# import the libraries
import numpy as np
import contractions
import re
from tqdm import tqdm
from keras import preprocessing, utils
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [64]:
# Function for preprocessing the given text
def preprocess_text(text):
    
    # Lowercase the text
    text = text.lower()
    
    # Decontracting the text (e.g. it's -> it is)
    text = contractions.fix(text)
    
    # Remove the punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    return text

In [65]:
# Preprocess the questions
questions_preprocessed = []
for i_question in tqdm(questions):
    questions_preprocessed.append(preprocess_text(i_question))
    
# Preprocess the answers
answers_preprocessed = []
for i_answer in tqdm(answers):
    answers_preprocessed.append(preprocess_text(i_answer))    

100%|██████████| 869/869 [00:00<00:00, 5944.53it/s]


In [66]:
# Take a look at the preprocessed questions and answers
for i in range(4):
    print("Question {}: \n".format(i), questions_preprocessed[i])
    print("")
    print("Answer {}: \n".format(i), answers_preprocessed[i])
    print("--------------------------------------------------------------------------")

Question 0: 
 have you read the communist

Answer 0: 
 yes  marx had made some interesting observations 
--------------------------------------------------------------------------
Question 1: 
 what is a government

Answer 1: 
 ideally it is a representative of the people 
--------------------------------------------------------------------------
Question 2: 
 what is greenpeace

Answer 2: 
 global organization promoting environmental activism 
--------------------------------------------------------------------------
Question 3: 
 what is capitalism

Answer 3: 
 the economic system in which all or most of the means of production and distribution  as land  factories  railroads  etc   are privately owned and operated for profit  originally under fully competitive conditions 
--------------------------------------------------------------------------


After preprocessing the dataset, we should add a start tag (e.g. `<START>`) and an end tag (e.g. `<END>`) to answers. Remember that we will only add these tags to answers and not questions. This requirement is because of the Seq2Seq model.

In [67]:
# Add <START> and <END> tag to each sentence
answers = list()
for i in range(len(answers_with_tags)):
    answers.append('<START> ' + answers_with_tags[i] + ' <END>')

In [68]:
answers[:5]

['<START> yes, marx had made some interesting observations. <END>',
 '<START> ideally it is a representative of the people. <END>',
 '<START> global organization promoting environmental activism. <END>',
 '<START> the economic system in which all or most of the means of production and distribution, as land, factories, railroads, etc., are privately owned and operated for profit, originally under fully competitive conditions. <END>',
 '<START> an established system of political administration by which a nation, state, district, etc. is governed. <END>']

Now it's time to tokenize our dataset. We use a class in Keras which allows us to vectorize a text corpus, by turning each text into either a sequence of integers (each integer being the index of a token in a dictionary) or into a vector where the coefficient for each token could be binary, based on word count, based on tf-idf, etc.


In [69]:
# Initialize the tokenizer
tokenizer = preprocessing.text.Tokenizer()

# Fit the tokenizer to questions and answers
tokenizer.fit_on_texts(questions + answers)

# Get the total vocab size
VOCAB_SIZE = len(tokenizer.word_index) + 1

print( 'VOCAB SIZE : {}'.format(VOCAB_SIZE))

VOCAB SIZE : 1975


In [70]:
### encoder input data

# Tokenize the questions
tokenized_questions = tokenizer.texts_to_sequences(questions)

# Get the length of longest sequence
maxlen_questions = max([len(x) for x in tokenized_questions])

# Pad the sequences
padded_questions = pad_sequences(tokenized_questions, maxlen=maxlen_questions, padding='post')

# Convert the sequences into array
encoder_input_data = np.array(padded_questions)

print(encoder_input_data.shape, maxlen_questions)

(869, 22) 22


In [71]:
### decoder input data

# Tokenize the answers
tokenized_answers = tokenizer.texts_to_sequences(answers)

# Get the length of longest sequence
maxlen_answers = max([len(x) for x in tokenized_answers])

# Pad the sequences
padded_answers = pad_sequences(tokenized_answers, maxlen=maxlen_answers, padding='post')

# Convert the sequences into array
decoder_input_data = np.array(padded_answers)

print(decoder_input_data.shape, maxlen_answers)

(869, 45) 45


In [72]:
### decoder_output_data

# Tokenize the answers
tokenized_answers = tokenizer.texts_to_sequences(answers)

# Iterate through index of tokenized answers
for i in range(len(tokenized_answers)) :

    #
    tokenized_answers[i] = tokenized_answers[i][1:]

# Pad the tokenized answers
padded_answers = pad_sequences(tokenized_answers, maxlen = maxlen_answers, padding = 'post')

# One hot encode
onehot_answers = utils.to_categorical(padded_answers, VOCAB_SIZE)

# Convert to numpy array
decoder_output_data = np.array(onehot_answers)

print(decoder_output_data.shape)

(869, 45, 1975)


In [73]:
# Saving all the arrays to storage
np.save("enc_in_data.npy", encoder_input_data)
np.save("dec_in_data.npy", decoder_input_data)
np.save("dec_tar_data.npy", decoder_output_data)

In [74]:
# Load all the arrays from storage
encoder_input_data = np.load("enc_in_data.npy")
decoder_input_data = np.load("dec_in_data.npy")
decoder_output_data = np.load("dec_tar_data.npy")

<br>

### 3.4. Train the Seq2Seq Model

---

In this section, we will use an architecture called Sequence to Sequence (or Seq2Seq). This model is used since the length of the input sequence (question) does not match the length of the output sequence (answer). This model is consists of an encoder and a decoder.
- __Encoder:__ <br> In this part of the network, we take the input data and train on it. Then we pass the last state of the recurrent layer to decoder. <br><br>
- __Decoder:__ <br> In this part of the network, we take the last state in encoder’s last recurrent layer. Then we will use it as an initial state in decoder's first recurrent layer.

<br>

<img src="assets/encoder_decoder.png">

<br>

Let's start by importing all the necessary libraries in Keras.

In [75]:
# Import the libraries
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model
from keras.optimizers import RMSprop
from keras.activations import softmax
from keras.callbacks import ModelCheckpoint

Below you can play around with hyperparameters for improving the model's accuracy.

In [76]:
# Hyper parameters
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 1e-3

In the following block of code, you will implement the Encoder. You can follow the below steps for creating the encoder: 

1.   Create an input for the Encoder.
2.   Create an embedding layer.
3.   Create an LSTM layer which also returns the states.
4.   Get the hidden state (state h) and cell state (state c) inside a list.

In [77]:
### Encoder Input

# Input for encoder
encoder_inputs = Input(shape = (None, ))

# Embedding layer
encoder_embedding = Embedding(input_dim = VOCAB_SIZE, output_dim = 200, mask_zero = True)(encoder_inputs)

# LSTM layer (that returns states in addition to output)
encoder_outputs, state_h, state_c = LSTM(units = 200, return_state = True)(encoder_embedding)

# Get the states for encoder
encoder_states = [state_h, state_c]

After creating your encoder, it's time to implement the decoder. You can follow the below steps for implementing the decoder:

1.   Create an input for the decoder.
2.   Create an embedding layer.
3.   Create an LSTM layer that returns states and sequences.
4.   Create a dense layer.
5.   Get the output.

In [78]:
### Decoder

# Input for decoder
decoder_inputs = Input(shape = (None,  ))

# Embedding layer
decoder_embedding = Embedding(input_dim = VOCAB_SIZE, output_dim = 200 , mask_zero = True)(decoder_inputs)

# LSTM layer (that returns states and sequences as well)
decoder_lstm = LSTM(units = 200 , return_state = True , return_sequences = True)

# Get the output of LSTM layer
decoder_outputs, _, _ = decoder_lstm(inputs = decoder_embedding, initial_state = encoder_states)

# Dense layer
decoder_dense = Dense(units = VOCAB_SIZE, activation = softmax) 

# Get the output of Dense layer
output = decoder_dense(decoder_outputs)

Now that you have implemented the encoder and decoder. It's time to create your model which takes two inputs: encoder's input and decoder's input. Then it outputs the decoder's output.

In [79]:
# Create the model
model = Model([encoder_inputs, decoder_inputs], output)

In [80]:
# Compile the model
model.compile(optimizer = RMSprop(lr = LEARNING_RATE), loss = "categorical_crossentropy")

In [81]:
# Summary
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 200)    395000      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 200)    395000      input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LS

In [82]:
# Train the model
model.fit(x = [encoder_input_data , decoder_input_data], 
          y = decoder_output_data, 
          batch_size = BATCH_SIZE, 
          epochs = EPOCHS) 

Epoch 1/50
869/869 [==============================] - 16s 18ms/step - loss: 6.1062
Epoch 2/50
869/869 [==============================] - 16s 18ms/step - loss: 5.3734
Epoch 3/50
869/869 [==============================] - 16s 19ms/step - loss: 5.1482
Epoch 4/50
869/869 [==============================] - 19s 21ms/step - loss: 4.9950
Epoch 5/50
869/869 [==============================] - 14s 16ms/step - loss: 4.8604
Epoch 6/50
869/869 [==============================] - 14s 16ms/step - loss: 4.7477
Epoch 7/50
869/869 [==============================] - 16s 19ms/step - loss: 4.6162
Epoch 8/50
869/869 [==============================] - 19s 22ms/step - loss: 4.5043
Epoch 9/50
869/869 [==============================] - 14s 17ms/step - loss: 4.3848
Epoch 10/50
869/869 [==============================] - 16s 18ms/step - loss: 4.2659
Epoch 11/50
869/869 [==============================] - 14s 16ms/step - loss: 4.1606
Epoch 12/50
869/869 [==============================] - 14s 17ms/step - loss: 4.0488
E

In [83]:
# Save the final model
model.save(filepath = './saved models/final_weight.h5') 
print("Model Weight Saved!")

/anaconda3/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 200) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 200) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


Model Weight Saved!


In [46]:
# Load the final model
model.load_weights('saved models/final_weight.h5') 
print("Model Weight Loaded!")

Model Weight Loaded!


<br>

### 3.5. Inference

---

Now it's time to use our model for inference. In other words, we will ask a question to our chatbot and it will answer us.

In [84]:
# Function for making inference
def make_inference_models():
    
    # Create a model that takes encoder's input and outputs the states for encoder
    encoder_model = Model(encoder_inputs, encoder_states)
    
    # Create two inputs for decoder which are hidden state (or state h) and cell state (or state c)
    decoder_state_input_h = Input(shape = (200, ))
    decoder_state_input_c = Input(shape = (200, ))
    
    # Store the two inputs for decoder inside a list
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    # Pass the inputs through LSTM layer you have created before
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding, initial_state = decoder_states_inputs)
    
    # Store the outputted hidden state and cell state from LSTM inside a list
    decoder_states = [state_h, state_c]

    # Pass the output from LSTM layer through the dense layer you have created before
    decoder_outputs = decoder_dense(decoder_outputs)

    # Create a model that takes decoder_inputs and decoder_states_inputs as inputs and outputs decoder_outputs and decoder_states
    decoder_model = Model([decoder_inputs] + decoder_states_inputs,
                          [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [85]:
# Function for converting strings to tokens
def str_to_tokens(sentence:str):

    # Lowercase the sentence and split it into words
    words = sentence.lower().split()

    # Initialize a list for tokens
    tokens_list = list()

    # Iterate through words
    for word in words:

        # Append the word index inside tokens list
        tokens_list.append(tokenizer.word_index[word]) 

    # Pad the sequences to be the same length
    return pad_sequences([tokens_list] , maxlen = maxlen_questions, padding = 'post')

In [95]:
# Initialize the model for inference
enc_model , dec_model = make_inference_models()

# Iterate through the number of times you want to ask question
for _ in range(5):

    # Get the input and predict it with the encoder model
    states_values = enc_model.predict(str_to_tokens(preprocess_text(input('Enter question : '))))

    # Initialize the target sequence with zero - array([[0.]])
    empty_target_seq = np.zeros(shape = (1, 1))

    # Update the target sequence with index of "start"
    empty_target_seq[0, 0] = tokenizer.word_index["start"]

    # Initialize the stop condition with False
    stop_condition = False

    # Initialize the decoded words with an empty string
    decoded_translation = ''

    # While stop_condition is false
    while not stop_condition :

        # Predict the (target sequence + the output from encoder model) with decoder model
        dec_outputs , h , c = dec_model.predict([empty_target_seq] + states_values)

        # Get the index for sampled word
        sampled_word_index = np.argmax(dec_outputs[0, -1, :])

        # Initialize the sampled word with None
        sampled_word = None

        # Iterate through words and their indexes
        for word, index in tokenizer.word_index.items() :

            # If the index is equal to sampled word's index
            if sampled_word_index == index :

                # Add the word to the decoded string
                decoded_translation += ' {}'.format(word)

                # Update the sampled word
                sampled_word = word
        
        # If sampled word is equal to "end" OR the length of decoded string is more that what is allowed
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:

            # Make the stop_condition to true
            stop_condition = True
            
        # Initialize back the target sequence to zero - array([[0.]])    
        empty_target_seq = np.zeros(shape = (1, 1))  

        # Update the target sequence with index of "start"
        empty_target_seq[0, 0] = sampled_word_index

        # Get the state values
        states_values = [h, c] 

    # Print the decoded string
    print(decoded_translation[:-3])

Enter question : Hello!
 hi 
Enter question : How are you doing?
 i am doing well 
Enter question : Can i ask you a question?
 sure ask away 
Enter question : What are your interests?
 i am interested in a computer 
Enter question : Tell me a joke
 what do you get when you cross a cat and a lemon 


In [96]:
# Iterate through the number of times you want to ask question
def text_to_text(input_text):
    
    # Initialize the model for inference
    enc_model , dec_model = make_inference_models()

    # Get the input and predict it with the encoder model
    states_values = enc_model.predict(str_to_tokens(preprocess_text(input_text)))

    # Initialize the target sequence with zero - array([[0.]])
    empty_target_seq = np.zeros(shape = (1, 1))

    # Update the target sequence with index of "start"
    empty_target_seq[0, 0] = tokenizer.word_index["start"]

    # Initialize the stop condition with False
    stop_condition = False

    # Initialize the decoded words with an empty string
    decoded_translation = ''

    # While stop_condition is false
    while not stop_condition :

        # Predict the (target sequence + the output from encoder model) with decoder model
        dec_outputs , h , c = dec_model.predict([empty_target_seq] + states_values)

        # Get the index for sampled word
        sampled_word_index = np.argmax(dec_outputs[0, -1, :])

        # Initialize the sampled word with None
        sampled_word = None

        # Iterate through words and their indexes
        for word, index in tokenizer.word_index.items() :

            # If the index is equal to sampled word's index
            if sampled_word_index == index :

                # Add the word to the decoded string
                decoded_translation += ' {}'.format(word)

                # Update the sampled word
                sampled_word = word
        
        # If sampled word is equal to "end" OR the length of decoded string is more that what is allowed
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:

            # Make the stop_condition to true
            stop_condition = True
            
        # Initialize back the target sequence to zero - array([[0.]])    
        empty_target_seq = np.zeros(shape = (1, 1))  

        # Update the target sequence with index of "start"
        empty_target_seq[0, 0] = sampled_word_index

        # Get the state values
        states_values = [h, c] 

    # return the decoded string
    return decoded_translation[:-3]

In [99]:
text_to_text("How are you doing?")

' i am doing well '

# 4. Text to Speech

---

In [104]:
# Import the libraries
import pyttsx3

In [105]:
# Construct a new TTS engine instance
engine = pyttsx3.init()

In [106]:
# Get all of the voices
voices = engine.getProperty('voices')

# Loop over voices and print their descriptions
for index, voice in enumerate(voices):
    print("Voice {}: ".format(index))
    print(" - ID: %s" % voice.id)
    print(" - Name: %s" % voice.name)
    print(" - Languages: %s" % voice.languages)
    print(" - Gender: %s" % voice.gender)
    print(" - Age: %s" % voice.age)
    print("")

Voice 0: 
 - ID: com.apple.speech.synthesis.voice.Alex
 - Name: Alex
 - Languages: ['en_US']
 - Gender: VoiceGenderMale
 - Age: 35

Voice 1: 
 - ID: com.apple.speech.synthesis.voice.alice
 - Name: Alice
 - Languages: ['it_IT']
 - Gender: VoiceGenderFemale
 - Age: 35

Voice 2: 
 - ID: com.apple.speech.synthesis.voice.alva
 - Name: Alva
 - Languages: ['sv_SE']
 - Gender: VoiceGenderFemale
 - Age: 35

Voice 3: 
 - ID: com.apple.speech.synthesis.voice.amelie
 - Name: Amelie
 - Languages: ['fr_CA']
 - Gender: VoiceGenderFemale
 - Age: 35

Voice 4: 
 - ID: com.apple.speech.synthesis.voice.anna
 - Name: Anna
 - Languages: ['de_DE']
 - Gender: VoiceGenderFemale
 - Age: 35

Voice 5: 
 - ID: com.apple.speech.synthesis.voice.carmit
 - Name: Carmit
 - Languages: ['he_IL']
 - Gender: VoiceGenderFemale
 - Age: 35

Voice 6: 
 - ID: com.apple.speech.synthesis.voice.damayanti
 - Name: Damayanti
 - Languages: ['id_ID']
 - Gender: VoiceGenderFemale
 - Age: 35

Voice 7: 
 - ID: com.apple.speech.synthesis.

In [107]:
### Voice properties    

# Speed percent (can go over 100)
engine.setProperty(name = 'rate', value = 180)    

# Volume 0-1
engine.setProperty(name = 'volume', value = 0.9)

# Voice ID
en_voice_id = "com.apple.speech.synthesis.voice.daniel.premium"
engine.setProperty('voice', en_voice_id)

In [108]:
# Convert the text to speech
engine.say("You've got mail!")
engine.say("The pyttsx3 module supports native Windows and Mac speech APIs but also supports espeak, making it the best available text-to-speech package.")
engine.runAndWait() 

<br>

# 5. Finalize your Conversational Based Agent

---

Now it's time to put everything together so you can do speech-to-text, text-to-text, and text-to-speech at the same time. For this, you will create a button which after pushing you can speak and your model will speck to you.

In [121]:
# Import the libraries 
import ipywidgets as widgets
from IPython.display import display

In [122]:
def agent():
    button = widgets.Button(description="Click Here for Talking!")
    output = widgets.Output()
    display(button, output)
    def on_button_clicked(b):
        with output:

            # Speech recognition
            text = speech_recognition_g()
            print(" - YOU SAID: ", text)

            # Text-to-text
            response = text_to_text(text)
            print(" + AGENT: ", response)

            # Text to speech
            engine.say(response)
            engine.runAndWait() 


    button.on_click(on_button_clicked)

In [123]:
agent()

Button(description='Click Here for Talking!', style=ButtonStyle())

Output()